# Naive Bayes Algorithm

A machine learning tutorial from https://machinelearningmastery.com/naive-bayes-classifier-scratch-python/

Reimplemented in Scala

## About Naive Bayes
The Naive Bayes algorithm is an intuitive method that uses the probabilities of each attribute belonging to each class to make a prediction. It is the supervised learning approach you would come up with if you wanted to model a predictive modeling problem probabilistically.

Naive bayes simplifies the calculation of probabilities by assuming that the probability of each attribute belonging to a given class value is independent of all other attributes. This is a strong assumption but results in a fast and effective method.

The probability of a class value given a value of an attribute is called the conditional probability. By multiplying the conditional probabilities together for each attribute for a given class value, we have a probability of a data instance belonging to that class.

To make a prediction we can calculate probabilities of the instance belonging to each class and select the class value with the highest probability.

Naive bases is often described using categorical data because it is easy to describe and calculate using ratios. A more useful version of the algorithm for our purposes supports numeric attributes and assumes the values of each numerical attribute are normally distributed (fall somewhere on a bell curve). Again, this is a strong assumption, but still gives robust results.

## Predict the Onset of Diabetes
The test problem we will use in this tutorial is the Pima Indians Diabetes problem.

This problem is comprised of 768 observations of medical details for Pima indians patents. The records describe instantaneous measurements taken from the patient such as their age, the number of times pregnant and blood workup. All patients are women aged 21 or older. All attributes are numeric, and their units vary from attribute to attribute.

Each record has a class value that indicates whether the patient suffered an onset of diabetes within 5 years of when the measurements were taken (1) or not (0).

This is a standard dataset that has been studied a lot in machine learning literature. A good prediction accuracy is 70%-76%.

## Naive Bayes Algorithm Tutorial

This tutorial is broken down into the following steps:

1. **Handle Data**: Load the data from CSV file and split it into training and test datasets.
2. **Summarize Data**: summarize the properties in the training dataset so that we can calculate probabilities and make predictions.
3. **Make a Prediction**: Use the summaries of the dataset to generate a single prediction.
4. **Make Predictions**: Generate predictions given a test dataset and a summarized training dataset.
5. **Evaluate Accuracy**: Evaluate the accuracy of predictions made for a test dataset as the percentage correct out of all predictions made.
6. **Tie it Together**: Use all of the code elements to present a complete and standalone implementation of the Naive Bayes algorithm.

In [2]:
import scala.collection.mutable.ArrayBuffer

def loadCsv(filename: String): List[List[Double]] = {
    val bufferedSource = io.Source.fromFile(filename)
    val rows = ArrayBuffer[List[Double]]()
    for (line <- bufferedSource.getLines) {
        rows += line.split(",").map(_.trim).map(_.toDouble).toList
    }
    bufferedSource.close
    rows.toList
}

import scala.collection.mutable.ArrayBuffer


defined function loadCsv

In [3]:
import util.Random

def splitDataset(dataset: List[List[Double]], splitRatio: Double): (List[List[Double]], List[List[Double]]) = {
    val shuffled = Random.shuffle(dataset)
    shuffled.splitAt((dataset.length * splitRatio).toInt)
}

import util.Random


defined function splitDataset

In [4]:
def separateByClass(dataset: List[List[Double]]): Map[Double, List[List[Double]]] = dataset.groupBy(_.last)

defined function separateByClass

In [5]:
// TODO: validate this implementation, is the python one better?
def calcMeanStd(x: List[Double]): (Double, Double) = {
    @scala.annotation.tailrec
    def meanStd(x: List[Double], mu: Double, Q: Double, count: Int): (Double, Double) = {
        if (count >= x.length) (mu, Math.sqrt(Q / x.length))
        else {
            val newCount = count + 1
            val newMu = x(count) / newCount + mu * (1.0 - 1.0 / newCount)
            val newQ = Q + (x(count) - mu) * (x(count) - newMu)
            meanStd(x, newMu, newQ, newCount)   
        }        
    }

    meanStd(x, 0.0, 0.0, 0)    
}

def mean(numbers: List[Double]): Double = calcMeanStd(numbers)._1
def stdev(numbers: List[Double]): Double = calcMeanStd(numbers)._2

defined function calcMeanStd
defined function mean
defined function stdev

In [6]:
def summarize(dataset: List[List[Double]]): List[(Double, Double)] = dataset.transpose.dropRight(1).map(calcMeanStd)

defined function summarize

In [7]:
def summarizeByClass(dataset: List[List[Double]]): Map[Double, List[(Double, Double)]] = separateByClass(dataset).map {
    case (a, b) => (a, summarize(b))
}

defined function summarizeByClass

In [8]:
import scala.math.{exp, pow, sqrt, Pi}
def calculateProbability(x: Double, mean: Double, stdev: Double): Double = {
    val exponent = exp(-(pow(x - mean, 2) / (2 * pow(stdev, 2))))
    (1 / (math.sqrt(2 * Pi) * stdev)) * exponent
}

import scala.math.{exp, pow, sqrt, Pi}

defined function calculateProbability

In [9]:
def calculateClassProbabilities(summaries: Map[Double, List[(Double, Double)]], inputVector: List[Double]) = {
    summaries.map{ case (classValue, classSummaries) => (classValue, classSummaries.foldLeft(1.0) {
        (acc, tup) => calculateProbability(inputVector.head, tup._1, tup._2)
    } ) }
}

defined function calculateClassProbabilities

In [10]:
def predict(summaries: Map[Double, List[(Double, Double)]], inputVector: List[Double]) = {
    calculateClassProbabilities(summaries, inputVector).maxBy(_._2)._1
}

defined function predict

In [11]:
def getPredictions(summaries: Map[Double, List[(Double, Double)]], inputVector: List[List[Double]]) = {
    for (iv <- inputVector) yield predict(summaries, iv)
}

defined function getPredictions

In [12]:
def getAccuracy(testSet: List[List[Double]], predictions: List[Double]): Double = {
    val correct = for ( (data: List[Double], prediction: Double) <- testSet zip predictions) yield data.last == prediction
    (correct.filter(_ == true).length / testSet.length.toDouble) * 100
}

defined function getAccuracy

In [13]:
val testSet = List(List(1.0, 1.0, 1.0, 0.0), List(2.0, 2.0, 2.0, 0.0), List(3.0, 3.0, 3.0, 1.0))
val predictions = List(0.0, 0.0, 0.0)
getAccuracy(testSet, predictions)

testSet: List[List[Double]] = List(
  List(1.0, 1.0, 1.0, 0.0),
  List(2.0, 2.0, 2.0, 0.0),
  List(3.0, 3.0, 3.0, 1.0)
)
predictions: List[Double] = List(0.0, 0.0, 0.0)
res12_2: Double = 66.66666666666666

In [24]:
def main() = {
    val filename = "pima-indians-diabetes.data.csv"
    val splitRatio = 0.67
    val dataset = loadCsv(filename)
    val (trainingSet, testSet) = splitDataset(dataset, splitRatio)
    print(s"Split ${dataset.length} rows into train=${trainingSet.length} and test=${testSet.length} rows\n")
    val summaries = summarizeByClass(trainingSet)
    val predictions = getPredictions(summaries, testSet)
    val accuracy = getAccuracy(testSet, predictions)
    print(s"Accuracy: $accuracy")
}

defined function main

In [25]:
main()

Split 768 rows into train=514 and test=254 rows
Accuracy: 68.11023622047244